<a href="https://colab.research.google.com/github/aguilarksd/BigData_Training/blob/master/Pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

# 1st

In [ ]:
# Import necessary libraries
from pyspark.sql import SparkSession

# Create a SparkSession
# This is the entry point for all PySpark applications
spark = SparkSession.builder.appName("WordCountColab").master("local[*]").getOrCreate()

# Create a sample text file in the Colab environment
# This simulates your D:/input/data.txt file
data = "Hello PySpark Hello World"
with open("data.txt", "w") as f:
    f.write(data)

# Read the text file into an RDD
rdd1 = spark.sparkContext.textFile("data.txt")

# Perform the word count
rdd2 = rdd1.flatMap(lambda line: line.split(" "))
rdd3 = rdd2.map(lambda word: (word, 1))
rdd4 = rdd3.reduceByKey(lambda a, b: a + b)

# Collect and print the results
results = rdd4.collect()
for w, c in results:
    print(f"Word: {w}, Count: {c}")

# Stop the SparkSession
spark.stop()

Word: PySpark, Count: 1
Word: Hello, Count: 2
Word: World, Count: 1


# 2nd

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from pyspark.sql.functions import col, trim, lower, upper, when, avg, first, rank
from pyspark.sql.window import Window


In [ ]:
spark = SparkSession.builder.appName("dfdemo").master("local[*]").getOrCreate()

Create a data frame

In [ ]:
productdf = spark.read.option("header",True).option("inferSchema",True).csv("drive/MyDrive/Data/products.csv")

In [ ]:
productdf.show()

+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+---------------+--------------------+------+
|product_number|        product_name|    product_category|    product_scale|product_manufacturer| product_description|         length|               width|height|
+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+---------------+--------------------+------+
|      S10_1678|1969 Harley David...|         Motorcycles|             1:10|     Min Lin Diecast|This replica feat...|           7933|               48.81|  95.7|
|      S10_1949|1952 Alpine Renau...|        Classic Cars|             1:10|Classic Metal Cre...|Turnable front wh...|           7305|               98.58| 214.3|
|      S10_2016|1996 Moto Guzzi 1...|         Motorcycles|             1:10|Highway 66 Mini C...|Official Moto Guz...|           6625|               68.99|118.94|
|      S10_4698|2003 H

create with other way

In [ ]:
product_schema = StructType([
    StructField("product_number", StringType(), True),
    StructField("product_name", StringType(), True),
    StructField("product_category", StringType(), True),
     StructField("product_scale", StringType(), True),
     StructField("product_Manufacturer", StringType(), True),
     StructField("product_description", StringType(), True),
     StructField("length", DoubleType(), True),
     StructField("width", DoubleType(), True),
    StructField("height", DoubleType(), True)
])

In [ ]:
productdf = spark.read.option("header",True).schema(product_schema).csv("drive/MyDrive/Data/products.csv")

In [ ]:
productdf.show(5)

+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+------+-----+------+
|product_number|        product_name|    product_category|    product_scale|product_Manufacturer| product_description|length|width|height|
+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+------+-----+------+
|      S10_1678|1969 Harley David...|         Motorcycles|             1:10|     Min Lin Diecast|This replica feat...|7933.0|48.81|  95.7|
|      S10_1949|1952 Alpine Renau...|        Classic Cars|             1:10|Classic Metal Cre...|Turnable front wh...|7305.0|98.58| 214.3|
|      S10_2016|1996 Moto Guzzi 1...|         Motorcycles|             1:10|Highway 66 Mini C...|Official Moto Guz...|6625.0|68.99|118.94|
|      S10_4698|2003 Harley-David...|         Motorcycles|             1:10|   Red Start Diecast|Model features, o...|3252.0|85.68| 136.0|
|      S10_2699| precision 

# Cleaning

In [ ]:
casteddf = productdf.withColumn("length", col("length").cast("int"))

In [ ]:
casteddf.dropDuplicates(["product_number"]).show(5)

+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+------+-----+------+
|product_number|        product_name|    product_category|    product_scale|product_Manufacturer| product_description|length|width|height|
+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+------+-----+------+
|      S10_1678|1969 Harley David...|         Motorcycles|             1:10|     Min Lin Diecast|This replica feat...|  7933|48.81|  95.7|
|      S10_1949|1952 Alpine Renau...|        Classic Cars|             1:10|Classic Metal Cre...|Turnable front wh...|  7305|98.58| 214.3|
|      S10_2016|1996 Moto Guzzi 1...|         Motorcycles|             1:10|Highway 66 Mini C...|Official Moto Guz...|  6625|68.99|118.94|
|      S10_2699| precision diecas...| baked enamel finish| 1:10 scale model|    removable fender| seat and tank co...|  5582|91.02|193.66|
|      S10_4698|2003 Harley

In [ ]:
casteddf.show(2)

+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+------+-----+------+
|product_number|        product_name|    product_category|    product_scale|product_Manufacturer| product_description|length|width|height|
+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+------+-----+------+
|      S10_1678|1969 Harley David...|         Motorcycles|             1:10|     Min Lin Diecast|This replica feat...|  7933|48.81|  95.7|
|      S10_1949|1952 Alpine Renau...|        Classic Cars|             1:10|Classic Metal Cre...|Turnable front wh...|  7305|98.58| 214.3|
|      S10_2016|1996 Moto Guzzi 1...|         Motorcycles|             1:10|Highway 66 Mini C...|Official Moto Guz...|  6625|68.99|118.94|
|      S10_4698|2003 Harley-David...|         Motorcycles|             1:10|   Red Start Diecast|Model features, o...|  3252|85.68| 136.0|
|      S10_2699| precision 

missin values

In [ ]:
casteddf = casteddf.na.fill("unknown",subset=["product_name","product_category"])
casteddf = casteddf.na.fill(0,subset=["length","width"])

In [ ]:
casteddf.withColumn("product_name",trim(lower(col("product_name"))))\
.withColumn("product_category",trim(upper(col("product_category")))).show(2)

+--------------+--------------------+----------------+-------------+--------------------+--------------------+------+-----+------+
|product_number|        product_name|product_category|product_scale|product_Manufacturer| product_description|length|width|height|
+--------------+--------------------+----------------+-------------+--------------------+--------------------+------+-----+------+
|      S10_1678|1969 harley david...|     MOTORCYCLES|         1:10|     Min Lin Diecast|This replica feat...|  7933|48.81|  95.7|
|      S10_1949|1952 alpine renau...|    CLASSIC CARS|         1:10|Classic Metal Cre...|Turnable front wh...|  7305|98.58| 214.3|
+--------------+--------------------+----------------+-------------+--------------------+--------------------+------+-----+------+
only showing top 2 rows



predicate push down

In [ ]:
validdf = casteddf.filter((col("length")>0) & (col("width")>0))

column pruning

In [ ]:
validdf.drop("product_description","product_scale").show(5)

+--------------+--------------------+--------------------+--------------------+------+-----+------+
|product_number|        product_name|    product_category|product_Manufacturer|length|width|height|
+--------------+--------------------+--------------------+--------------------+------+-----+------+
|      S10_1678|1969 Harley David...|         Motorcycles|     Min Lin Diecast|  7933|48.81|  95.7|
|      S10_1949|1952 Alpine Renau...|        Classic Cars|Classic Metal Cre...|  7305|98.58| 214.3|
|      S10_2016|1996 Moto Guzzi 1...|         Motorcycles|Highway 66 Mini C...|  6625|68.99|118.94|
|      S10_4698|2003 Harley-David...|         Motorcycles|   Red Start Diecast|  3252|85.68| 136.0|
|      S10_2699| precision diecas...| baked enamel finish|    removable fender|  5582|91.02|193.66|
+--------------+--------------------+--------------------+--------------------+------+-----+------+
only showing top 5 rows



In [ ]:
validdf = validdf.withColumn("product_size", when(col("length")<1000,"small")\
.when(col("length")>=5000,"medium")\
.otherwise("large"))

In [ ]:
validdf.show(5)

+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+------+-----+------+------------+
|product_number|        product_name|    product_category|    product_scale|product_Manufacturer| product_description|length|width|height|product_size|
+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+------+-----+------+------------+
|      S10_1678|1969 Harley David...|         Motorcycles|             1:10|     Min Lin Diecast|This replica feat...|  7933|48.81|  95.7|      medium|
|      S10_1949|1952 Alpine Renau...|        Classic Cars|             1:10|Classic Metal Cre...|Turnable front wh...|  7305|98.58| 214.3|      medium|
|      S10_2016|1996 Moto Guzzi 1...|         Motorcycles|             1:10|Highway 66 Mini C...|Official Moto Guz...|  6625|68.99|118.94|      medium|
|      S10_4698|2003 Harley-David...|         Motorcycles|             1:10|   Red Start

In [ ]:
validdf.groupBy("product_size").avg("length").show(4)

+------------+------------------+
|product_size|       avg(length)|
+------------+------------------+
|      medium| 7584.773584905661|
|       small|480.15384615384613|
|       large|            2926.0|
+------------+------------------+



In [ ]:
validdf.groupBy("product_size").agg(avg("length").alias("avgLen"),avg("width").alias("avgwid")).show(4)

+------------+------------------+-----------------+
|product_size|            avgLen|           avgwid|
+------------+------------------+-----------------+
|      medium| 7584.773584905661|55.09547169811321|
|       small|480.15384615384613|47.08846153846154|
|       large|            2926.0|           56.165|
+------------+------------------+-----------------+



In [ ]:
validdf.groupBy("product_category").pivot("product_size").agg(first("product_name")).show(4)

+--------------------+--------------------+--------------------+--------------------+
|    product_category|               large|              medium|               small|
+--------------------+--------------------+--------------------+--------------------+
| baked enamel finish|                NULL| precision diecas...|                NULL|
|        Classic Cars| 1972 Alfa Romeo GTA|1952 Alpine Renau...|   1968 Ford Mustang|
|         Motorcycles|2003 Harley-David...|1969 Harley David...|1960 BSA Gold Sta...|
|              Planes|1928 British Roya...|1980s Black Hawk ...|      P-51-D Mustang|
+--------------------+--------------------+--------------------+--------------------+
only showing top 4 rows



ranking

In [ ]:
window1 = Window.partitionBy("product_category").orderBy(col("length").desc())
validdf.withColumn("rank",rank().over(window1)).show(5)

+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+------+-----+------+------------+----+
|product_number|        product_name|    product_category|    product_scale|product_Manufacturer| product_description|length|width|height|product_size|rank|
+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+------+-----+------+------------+----+
|      S10_2699| precision diecas...| baked enamel finish| 1:10 scale model|    removable fender| seat and tank co...|  5582|91.02|193.66|      medium|   1|
|      S18_1984|    1995 Honda Civic|        Classic Cars|             1:18|     Min Lin Diecast|This model featur...|  9772|93.89|142.25|      medium|   1|
|      S24_3432| 2002 Chevy Corvette|        Classic Cars|             1:24|Gearbox Collectibles|The operating par...|  9446|62.11|107.08|      medium|   2|
|      S18_3482|1976 Ford Gran To...|        Classic Cars|

In [ ]:
validdf = validdf.withColumn("rank",rank().over(window1))

In [ ]:
validdf.filter(col("rank")==1).show(2)

+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+------+-----+------+------------+----+
|product_number|        product_name|    product_category|    product_scale|product_Manufacturer| product_description|length|width|height|product_size|rank|
+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+------+-----+------+------------+----+
|      S10_2699| precision diecas...| baked enamel finish| 1:10 scale model|    removable fender| seat and tank co...|  5582|91.02|193.66|      medium|   1|
|      S18_1984|    1995 Honda Civic|        Classic Cars|             1:18|     Min Lin Diecast|This model featur...|  9772|93.89|142.25|      medium|   1|
+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+------+-----+------+------------+----+
only showing top 2 rows



In [ ]:
validdf.show(5)

+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+------+-----+------+------------+----+
|product_number|        product_name|    product_category|    product_scale|product_Manufacturer| product_description|length|width|height|product_size|rank|
+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+------+-----+------+------------+----+
|      S10_2699| precision diecas...| baked enamel finish| 1:10 scale model|    removable fender| seat and tank co...|  5582|91.02|193.66|      medium|   1|
|      S18_1984|    1995 Honda Civic|        Classic Cars|             1:18|     Min Lin Diecast|This model featur...|  9772|93.89|142.25|      medium|   1|
|      S24_3432| 2002 Chevy Corvette|        Classic Cars|             1:24|Gearbox Collectibles|The operating par...|  9446|62.11|107.08|      medium|   2|
|      S18_3482|1976 Ford Gran To...|        Classic Cars|

In [ ]:
# len >1000 and width >1000 - large and wide
# len >1000 and width <1000 - large and narrow
# len <=1000 and width >1000 - small and wide
# otherwise - small and narrow
validdf.withColumn("product_size",
  when((col("length") > 1000) & (col("width") > 1000),"large and wide")\
  .when((col("length") > 1000) & (col("width") < 1000),"large and narrow")\
  .when((col("length") <= 1000) & (col("width") > 1000),"small and wide")\
  .otherwise("small and narrow ")
).show(5)


+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+------+-----+------+----------------+----+
|product_number|        product_name|    product_category|    product_scale|product_Manufacturer| product_description|length|width|height|    product_size|rank|
+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+------+-----+------+----------------+----+
|      S10_2699| precision diecas...| baked enamel finish| 1:10 scale model|    removable fender| seat and tank co...|  5582|91.02|193.66|large and narrow|   1|
|      S18_1984|    1995 Honda Civic|        Classic Cars|             1:18|     Min Lin Diecast|This model featur...|  9772|93.89|142.25|large and narrow|   1|
|      S24_3432| 2002 Chevy Corvette|        Classic Cars|             1:24|Gearbox Collectibles|The operating par...|  9446|62.11|107.08|large and narrow|   2|
|      S18_3482|1976 Ford Gran To.